```
class LagouSpider(CrawlSpider):
    name = 'lagou'
    allowed_domains = ['www.lagou.com']
    start_urls = ['https://www.lagou.com']

    rules = (
        Rule(LinkExtractor(allow=("zhaopin/.*",)), follow=True),
        Rule(LinkExtractor(allow=("gongsi/j\d+.html",)), follow=True),
        Rule(LinkExtractor(allow=r'jobs/\d+.html'), callback='parse_job', follow=True),
    )
    
    def parse_job(self, response):
        pass
```

```
class Rule(object):
    def __init__(self, link_extractor, callback=None, cb_kwargs=None, follow=None, process_links=None, process_request=identity):
        pass
```

1. 这里可以看到,Rule类的主要传递参数为
    * link_extractor(链接提取器,提取那些需要爬取的网址)
    * callback=None(回调函数)
    * follow=None:是否跟进链接,若为True表示跟进('跟进'的意思是对该链接对应网页下的子链接进行同样的操作),如果Rule没有定义回调函数的话follow默认为True
2. Rule的回调函数参数的定义和普通Spider定义方式相同
```
def parse_job(self, response):
        #解析拉勾网的职位
        item_loader = LagouJobItemLoader(item=LagouJobItem(), response=response)
        item_loader.add_css("title", ".job-name::attr(title)")
        item_loader.add_value("url_object_id", get_md5(response.url))
        item_loader.add_xpath("job_city", "//*[@class='job_request']/p/span[2]/text()")
        item_loader.add_css("job_advantage", ".job-advantage p::text")
        item_loader.add_value("crawl_time", datetime.now())

        job_item = item_loader.load_item()

        return job_item
```

```
class LxmlLinkExtractor(FilteringLinkExtractor):
    def __init__(self, allow=(), deny=(), allow_domains=(), deny_domains=(), restrict_xpaths=(),
                 tags=('a', 'area'), attrs=('href',), canonicalize=False,
                 unique=True, process_value=None, deny_extensions=None, restrict_css=(),
                 strip=True):
```

1. 这里可以看到,LinkExtractor类的主要传递参数为
    * allow=():是一个turple,指定允许爬取的网址,传递的是正则表达式(deny=()作用和allow()相反)
    * allow_domains=():是一个turple,指定允许爬取的域名,传递的是正则表达式(deny_domains=()作用和allow_domains=()相反)
    * restrict_xpaths=()/restrict_css=():使用xpath/css表达式，和allow共同作用过滤链接.
        * rules = [Rule(LinkExtractor(allow = ('/job/list/.*',),restrict_xpaths=('//a[@number="numbers last"]')), callback='parse_item')]# 提取下一页中的链接，跟进到下一页